In [1]:
import torch
import os
import sys

import numpy as np

cwd = os.getcwd()
parent_dir = os.path.dirname(cwd)
sys.path.append(parent_dir)

from DataPipeline.preprocessing import node_encoder, tensor_to_smiles
from generation import Sampling_Path_Batch
from models import Model_GNNs

from dataclasses import dataclass

In [2]:
@dataclass
class Experiment:
    exp_name: str
    encod: str
    keku: bool
    train: bool
    encoding_size: int = 13
    edge_size: int = 3
    encoding_option: str = 'charged'
    compute_lambdas: bool = False

exp = Experiment('GNN_baseline_3_modif', 'charged', True, False, 13, 3, 'charged', False)
exp2 = Experiment('GNN_baseline_3_modif_debiased', 'charged', True, False, 13, 3, 'charged', False)

In [3]:
from rdkit import Chem
from rdkit.Chem import Descriptors

def logP(smiles_list):
    logP_values = []
    for smiles in smiles_list:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            logP_values.append(0)
        else: 
            logp = Descriptors.MolLogP(mol)
            if logp > 2.0 and logp < 2.5:
                logP_values.append(1)
            else:
                logP_values.append(0)
    return torch.tensor(logP_values)

def QED(smiles_list):
    qed_values = []
    for smiles in smiles_list:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            qed_values.append(0)
        else: 
            qed = Descriptors.qed(mol)
            if qed > 0.90:
                qed_values.append(1)
            else:
                qed_values.append(0)
    return torch.tensor(qed_values)

In [4]:
GNNs_q = Model_GNNs(exp)
GNNs_a = Model_GNNs(exp2)
GNNs_pi = Model_GNNs(exp2)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
Module_Gen = Sampling_Path_Batch(GNNs_q, GNNs_a, GNNs_pi, features = {'logP' : logP, 'QED' : QED}, lambdas = torch.Tensor([1.0, 1.0]), device = device, batch_size = 1000, args=exp)


Loading best checkpoint number 1 of the epoch 2050.0 with a loss of 0.5107812829972882
Loading best checkpoint number 1 of the epoch 2550.0 with a loss of 0.1984780294417413
Loading best checkpoint number 2 of the epoch 2700.0 with a loss of 10.3904066518488
..\trained_models\GNN_baseline_3_modif\GNN1_baseline\history_training\checkpoint_1.pt ..\trained_models\GNN_baseline_3_modif\GNN2_baseline\history_training\checkpoint_1.pt ..\trained_models\GNN_baseline_3_modif\GNN3_split_two_without_node_embedding\history_training\checkpoint_2.pt
Loading best checkpoint number 1 of the epoch 2950.0 with a loss of 0.5103764619447081
Loading best checkpoint number 2 of the epoch 2150.0 with a loss of 0.19804639930989767
Loading best checkpoint number 2 of the epoch 2700.0 with a loss of 10.3904066518488
..\trained_models\GNN_baseline_3_modif_debiased\GNN1_charged_baseline_debiased\history_training\checkpoint_1.pt ..\trained_models\GNN_baseline_3_modif_debiased\GNN2_charged_baseline_debiased\history_

In [5]:
def compute_optimal_lambdas(Module_Gen, desired_moments, sample_size=5, n_iters=1000, lr=.5, min_nabla_lambda = 0.001): #how do they define the learning rate and sample size maybe do more
        """
        This performs the first step: Constraints --> EBM through self-normalized importance sampling. 
        Args:
            sample_size: total number of samples to use for lambda computation
        Returns:
            dicitonary of optimal lambdas per constraint: {'black': lambda_1, 'positive': lambda_2}
        """


        print("Computing Optimal Lambdas for desired moments...")

        max_n_iters = n_iters

        feature_names = list(Module_Gen.features.keys())
        mu_star = desired_moments #name mu_bar in the pseudo code

        mu_star = torch.tensor([mu_star[f] for f in feature_names])
        lambdas = Module_Gen.lambdas.cpu()

        # Collect sample_size samples for this:
        list_feature_tensor = []
        for i in range(sample_size):
            #if we do multi processing, i think here the best 
            #put pi without grad for lambdas

            Module_Gen.full_generation()
            Module_Gen.convert_to_smiles()
            Module_Gen.compute_features()

            q_value = Module_Gen.q_value
            a_value = Module_Gen.a_value
            pi_value = Module_Gen.pi_value

            batch_features_values = Module_Gen.all_features_values

            
            list_feature_tensor.append(batch_features_values)

        all_feature_tensor = torch.cat(list_feature_tensor, dim=0)  # [sample_sz*size_batch x F]

        #### check for zero-occuring features. 
        # If a constraint has not occurred in your sample, no lambdas will be learned for that constraint, so we must check.

        for i, feature  in enumerate(feature_names):
            assert all_feature_tensor[:, i].sum().item() > 0, "constraint {feature} hasn't occurred in the samples, use a larger sample size"

        for step in range(max_n_iters): #SGD for finding lambdas

            # 1. calculate P_over_q batch wise with current lambdas which will be name w
            ## compute new exponents

            w = torch.exp(torch.matmul(all_feature_tensor, lambdas.to(all_feature_tensor.device)))


            # 2. compute mu (mean) of features given the current lambda using SNIS
            mu_lambda_numerator = w.view(1, -1).matmul(all_feature_tensor ).squeeze(0) # F
            mu_lambda_denominator = w.sum()
            mu_lambda = mu_lambda_numerator / mu_lambda_denominator # F

            # 3. Update current Lambdas
            nabla_lambda = mu_star - mu_lambda.cpu()
            err = np.linalg.norm(nabla_lambda.cpu().numpy())
            print("step: %s \t ||nabla_lambda|| = %.6f" %(step, err))
            lambdas = lambdas + lr * nabla_lambda
            print("\tlambdas : {} ".format(Module_Gen.lambdas))
            print("\tμ: {}".format(mu_lambda))
            print("\tμ*: {}".format(mu_star))

            Module_Gen.lambdas = lambdas
            
            ## Check if error is less than tolerance, then break.
            if err < min_nabla_lambda: 
                break

In [6]:
compute_optimal_lambdas(Module_Gen, {'logP': 1.0, 'QED': 1.0}, sample_size=5, n_iters=1000, lr=.5, min_nabla_lambda = 0.001)

Computing Optimal Lambdas for desired moments...
step: 0 	 ||nabla_lambda|| = 1.074768
	lambdas : tensor([1., 1.], device='cuda:0') 
	μ: tensor([0.2877, 0.1951], device='cuda:0')
	μ*: tensor([1., 1.])
step: 1 	 ||nabla_lambda|| = 0.956968
	lambdas : tensor([1.3561, 1.4024]) 
	μ: tensor([0.3754, 0.2750], device='cuda:0')
	μ*: tensor([1., 1.])
step: 2 	 ||nabla_lambda|| = 0.834391
	lambdas : tensor([1.6684, 1.7649]) 
	μ: tensor([0.4618, 0.3624], device='cuda:0')
	μ*: tensor([1., 1.])
step: 3 	 ||nabla_lambda|| = 0.719039
	lambdas : tensor([1.9376, 2.0837]) 
	μ: tensor([0.5394, 0.4479], device='cuda:0')
	μ*: tensor([1., 1.])
step: 4 	 ||nabla_lambda|| = 0.618327
	lambdas : tensor([2.1679, 2.3598]) 
	μ: tensor([0.6049, 0.5244], device='cuda:0')
	μ*: tensor([1., 1.])
step: 5 	 ||nabla_lambda|| = 0.534311
	lambdas : tensor([2.3654, 2.5976]) 
	μ: tensor([0.6583, 0.5892], device='cuda:0')
	μ*: tensor([1., 1.])
step: 6 	 ||nabla_lambda|| = 0.465704
	lambdas : tensor([2.5363, 2.8030]) 
	μ: tenso